## CS310 Natural Language Processing
## Assignment 4. Dependency Parsing

**Total points**: 50

In this assignment, you will train feed-forward neural network-based dependency parser and evaluate its performance on the provided treebank dataset.

### 0. Import Necessary Libraries

In [154]:
import torch.nn as nn
from dep_utils import conll_reader, DependencyTree
import copy
from pprint import pprint
from collections import Counter, defaultdict
from typing import List, Dict, Tuple
import numpy as np
import torch
import torch.nn.functional as F
from collections import Counter
from torch.utils.data import DataLoader, Dataset
import optim


### 1. Read Data and Generate Training Instances

In [155]:
print('In train.conll:')
with open('valid_data/valid_train.conll') as f:
    train_trees = list(conll_reader(f))
print(f'{len(train_trees)} trees read.')

print('In dev.conll:')
with open('valid_data/valid_dev.conll') as f:
    dev_trees = list(conll_reader(f))
print(f'{len(dev_trees)} trees read.')

print('In test.conll:')
with open('valid_data/valid_test.conll') as f:
    test_trees = list(conll_reader(f))
print(f'{len(test_trees)} trees read.')

In train.conll:
39712 trees read.
In dev.conll:
1695 trees read.
In test.conll:
2408 trees read.


In [156]:
# Re-use the code from Lab 7
class RootDummy(object):
    def __init__(self):
        self.head = None
        self.id = 0
        self.deprel = None
    def __repr__(self):
        return "<ROOT>"

class State(object):
    def __init__(self, sentence=[],words=[]):
        self.stack = []
        self.buffer = []
        self.stack_words = []
        self.buffer_words = []
        if sentence:
            self.buffer = list(reversed(sentence))
        if words:
            self.buffer_words = list(reversed(words))
        self.deps = set()

    def shift(self):
        ### START YOUR CODE ###
        if self.buffer:
            front_of_buffer=self.buffer[-1]
            self.buffer.pop()
            self.stack.append(front_of_buffer)

            front_of_buffer_word=self.buffer_words[-1]
            self.buffer_words.pop()
            self.stack_words.append(front_of_buffer_word)

        ### END YOUR CODE ###

    def left_arc(self, label: str):
        assert len(self.stack) >= 2
        ### START YOUR CODE ###

        s1=self.stack[-1]
        s2 = self.stack[-2]  

        s1_word=self.stack_words[-1]
        s2_word = self.stack_words[-2]  
        

        self.stack.pop(-2)
        self.stack_words.pop(-2)

        self.deps.add((s1, s2, label))
        ### END YOUR CODE ###

    def right_arc(self, label: str):
        assert len(self.stack) >= 2
        ### START YOUR CODE ###
        s1=self.stack[-1]
        s2 = self.stack[-2] 

        s1_word=self.stack_words[-1]
        s2_word = self.stack_words[-2] 
        
        self.stack.pop()
        self.stack_words.pop()

        self.deps.add((s2, s1, label))
        ### END YOUR CODE ###

    def __repr__(self):
        return "({},{},{},{},{})".format(self.stack, self.buffer, self.deps,self.stack_words,self.buffer_words)

def get_training_instances(dep_tree) -> List[Tuple[State, Tuple[str, str]]]:
    deprels = dep_tree.deprels

    word_ids = list(deprels.keys())
    words_list=[]
    words = list(deprels.values())
    for i in words:
        words_list.append((i.word,i.pos))
    state = State(word_ids,words_list)
    state.stack.append(0) # ROOT
    state.stack_words.append(("<ROOT>",'<ROOT>')) # ROOT

    childcount = defaultdict(int)
    for _, rel in deprels.items():
        childcount[rel.head] += 1

    seq = []


    dep_relation=[]
    for i in range(1,len(deprels)+1):
        dep_relation.append((deprels[i].head,i))
    

    while len(state.buffer) > 0 or len(state.stack) > 1:
    

        if state.stack[-1] == 0:
            seq.append((copy.deepcopy(state), ("shift", None)))
            state.shift()
            continue
        
        stack_top1 = deprels[state.stack[-1]]
        if state.stack[-2] == 0:
            stack_top2 = RootDummy()
        else:
            stack_top2 = deprels[state.stack[-2]]

        ### START YOUR CODE ###

        if (int(state.stack[-1]), int(state.stack[-2])) in dep_relation:
            relation = stack_top2.deprel
            
            action = "left_arc"
            
            seq.append((copy.deepcopy(state), (action, relation)))
            childcount[state.stack[-1]] -= 1 
            state.left_arc(relation)
        elif (int(state.stack[-2]), int(state.stack[-1])) in dep_relation and childcount[state.stack[-1]] == 0:
            relation = stack_top1.deprel
            
            action = "right_arc"
            
            seq.append((copy.deepcopy(state), (action, relation)))
            childcount[state.stack[-2]] -= 1
            state.right_arc(relation)
        else:
            
            seq.append((copy.deepcopy(state), ("shift", None)))
            state.shift()
            
        ### END YOUR CODE ###
    
    seq.append((copy.deepcopy(state), ("done", None)))

    return seq


In [158]:

words_set=[]
tags_set=[]
actions_set=[]
for tree in train_trees:
    words_set.extend(tree.words())
    tags_set.extend(tree.pos())
    
    for i in tree.deprels:
        actions_set.append(tree.deprels[i].deprel)

words_set=set(words_set)
tags_set=set(tags_set)
actions_set=set(actions_set)


def create_action_vocab():
    action_vocab=[]

    for i in actions_set:
        if i !='root':
            action_vocab.append(('left_arc',i))
            action_vocab.append(('right_arc',i))
        else:
            action_vocab.append(('right_arc',i))

    
    action_vocab.append(('shift', None))
    
    return action_vocab

action_vocab=create_action_vocab()
print(len(words_set))
print(len(tags_set))
print(len(actions_set))
print(len(action_vocab))


44307
46
39
78


### 1) Implement the Feature Extractor (10 points) 


In [159]:
DIMENSION=100
NUM_FEATURES = 12  

In [160]:


class BagOfWordsEmbedding:
    def __init__(self, dimension):
        self.vocab = {}
        self.vocab_size = 0
        self.dimension = dimension

    def build_vocab(self, corpus):
        """Build vocabulary."""
        word_counts = Counter()
        for document in corpus:
            if document is not None:
                word_counts.update(document.split())
        self.vocab = {'<NULL>': 0, '<ROOT>': 1}
        idx = 2
        for word, _ in word_counts.items():
            self.vocab[word] = idx
            idx += 1
        self.vocab_size = len(self.vocab)

    def text_to_bow_vector(self, text):
        """Convert text to a bag-of-words vector."""
        if text is None:
            return np.zeros(self.dimension)
        bow_vector = np.zeros(self.dimension)
        words = text.split()
        for word in words:
            if word in self.vocab:
                bow_vector[self.vocab[word] % self.dimension] += 1
        return bow_vector



words_embedding = BagOfWordsEmbedding(DIMENSION)
words_embedding.build_vocab(words_set)
tags_embedding = BagOfWordsEmbedding(DIMENSION)
tags_embedding.build_vocab(tags_set)


word = "<ROOT>"
embedding1 = words_embedding.text_to_bow_vector(word)
print(embedding1)
word = "<NULL>"
embedding2 = tags_embedding.text_to_bow_vector(word)
print(embedding2)


[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]


In [191]:


class FeatureExtractor:
    def __init__(self, embedding_size, words_embedding, tags_embedding):
        self.embedding_size = embedding_size
        self.word_embeddings = words_embedding
        self.tag_embeddings = tags_embedding

    def get_word_embedding(self, word):
        return self.word_embeddings.text_to_bow_vector(word)

    def get_tag_embedding(self, tag):
        return self.tag_embeddings.text_to_bow_vector(tag)

    def extract_features(self, stack, buffer):
        features = []
        stack_extended = []
        buffer_extended = []
        stack_pos_extended = []
        buffer_pos_extended = []

        while len(stack) < 3:
            stack.insert(0,("<NULL>", "<NULL>"))
        while len(buffer) < 3:
            buffer.append(("<NULL>", "<NULL>"))

        for i in range(3):
            stack_word, stack_tag = stack[i]
            buffer_word, buffer_tag = buffer[i]

            stack_extended.append(stack_word)
            stack_pos_extended.append(stack_tag)
            buffer_extended.append(buffer_word)
            buffer_pos_extended.append(buffer_tag)

        for word in stack_extended + buffer_extended:
            features.extend(self.get_word_embedding(word))
        for tag in stack_pos_extended + buffer_pos_extended:
            features.extend(self.get_tag_embedding(tag))

        return np.array(features), stack_extended + buffer_extended + stack_pos_extended + buffer_pos_extended

fe = FeatureExtractor(DIMENSION, words_embedding, tags_embedding)
stack = [("<ROOT>", "<ROOT>"), ("the", "DT")]
buffer = [("apple", "NN"), ("trees", "NNS"), ("grow", "VB")]
features, feature_words = fe.extract_features(stack, buffer)
print(features.shape)  
print(features)
print(feature_words)


(1200,)
[1. 0. 0. ... 0. 0. 0.]
['<NULL>', '<ROOT>', 'the', 'apple', 'trees', 'grow', '<NULL>', '<ROOT>', 'DT', 'NN', 'NNS', 'VB']


### 2) Implement the scoring function (5 points)

In [192]:


class ScoringFunction(nn.Module):
    def __init__(self, input_size, hidden_size, num_actions):
        super(ScoringFunction, self).__init__()
        self.hidden = nn.Linear(input_size, hidden_size)
        self.output = nn.Linear(hidden_size, num_actions)

    def forward(self, x):
        x = self.hidden(x)  
        x = torch.tanh(x)  
        x = self.output(x) 
        return F.softmax(x, dim=-1)  


input_size = DIMENSION * NUM_FEATURES  
hidden_size = 200 
num_actions = len(action_vocab) 

mlp = ScoringFunction(input_size, hidden_size, num_actions)

phi_c = torch.randn(1, input_size)  
print("Feature vector size:", phi_c.size())

scores = mlp(phi_c)
print("Scores for each action:", scores)
print("Scores size:", scores.size())


Feature vector size: torch.Size([1, 1200])
Scores for each action: tensor([[0.0111, 0.0102, 0.0085, 0.0176, 0.0066, 0.0112, 0.0126, 0.0291, 0.0107,
         0.0129, 0.0168, 0.0105, 0.0089, 0.0152, 0.0177, 0.0156, 0.0112, 0.0103,
         0.0134, 0.0103, 0.0142, 0.0143, 0.0137, 0.0149, 0.0098, 0.0101, 0.0088,
         0.0086, 0.0135, 0.0226, 0.0106, 0.0102, 0.0121, 0.0116, 0.0088, 0.0123,
         0.0155, 0.0089, 0.0139, 0.0112, 0.0158, 0.0143, 0.0085, 0.0085, 0.0118,
         0.0116, 0.0103, 0.0127, 0.0195, 0.0109, 0.0083, 0.0126, 0.0208, 0.0145,
         0.0129, 0.0135, 0.0123, 0.0096, 0.0108, 0.0133, 0.0134, 0.0149, 0.0136,
         0.0085, 0.0179, 0.0163, 0.0119, 0.0080, 0.0134, 0.0189, 0.0115, 0.0147,
         0.0107, 0.0128, 0.0121, 0.0165, 0.0112, 0.0151]],
       grad_fn=<SoftmaxBackward0>)
Scores size: torch.Size([1, 78])


### 2. Build the Model

### 3. Train and Evaluate

### 3) Implement the Training Step (15 points)

In [193]:


class Parser(nn.Module):
    def __init__(self, embedding_size, hidden_size, num_actions):
        super(Parser, self).__init__()
        self.feature_extractor = FeatureExtractor(embedding_size, words_embedding, tags_embedding)
        self.scoring_function = ScoringFunction(embedding_size * 12, hidden_size, num_actions)

    
    def forward(self, X):

        scores_list = []
        for instance in X:
            cur_scores = self.scoring_function(instance)
                
            scores_list.append(cur_scores)
            
        scores = torch.stack(scores_list)
        return scores

    def compute_loss(self, predicted_scores, y):
        loss = F.cross_entropy(predicted_scores, y)
        return loss
    



def process(dep_trees: List[DependencyTree], word_vocab: dict, pos_vocab: dict, action_vocab, words_embedding, tags_embedding) -> List[Tuple[torch.Tensor, torch.Tensor]]:
    tensor_data = []
    
    for tree in dep_trees:
        instances = get_training_instances(tree)
      
        for state, action in instances:
            if action==('done', None):
                continue
            feature_extractor = FeatureExtractor(DIMENSION, words_embedding, tags_embedding)
            feature_vector, _ = state_to_feature_vector(state, feature_extractor)

           
            action_tensor = torch.tensor(action_vocab.index(action), dtype=torch.long)

            tensor_data.append({'X': feature_vector, 'y': action_tensor})

    return tensor_data

def state_to_feature_vector(state, feature_extractor):
    stack = state.stack_words
    buffer = state.stack_words
   
    
    features, feature_words = feature_extractor.extract_features(stack, buffer)

    feature_tensor = torch.tensor(features, dtype=torch.float32)

    return feature_tensor, feature_words





In [194]:
train_data = process(train_trees,words_set,tags_set,action_vocab,words_embedding, tags_embedding)
print("train_data process finish")
dev_data = process(dev_trees,words_set,tags_set,action_vocab,words_embedding, tags_embedding)
print("dev_data process finish")

train_data process finish
dev_data process finish


In [195]:
def train_model(model, train_loader, dev_loader, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train() 
        total_loss = 0.0
        for batch in train_loader:
            X_batch, y_batch = batch['X'], batch['y']
            optimizer.zero_grad() 
            predicted_scores = model(X_batch)
            loss = model.compute_loss(predicted_scores, y_batch)
            loss.backward()  
            optimizer.step()  
            total_loss += loss.item() * X_batch.size(0)
        train_loss = total_loss / len(train_loader.dataset)
        
        model.eval()  
        total_loss = 0.0
        for batch in dev_loader:
            X_batch, y_batch = batch['X'], batch['y']
            predicted_scores = model(X_batch)
            loss = model.compute_loss(predicted_scores, y_batch)
            total_loss += loss.item() * X_batch.size(0)
        dev_loss = total_loss / len(dev_loader.dataset)
        
        print(f'Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}, Dev Loss: {dev_loss:.4f}')





In [196]:

learning_rate=0.001
batch_size=32
num_epochs=1
hidden_size = 200
num_actions = len(action_vocab)
model = Parser(DIMENSION, hidden_size, num_actions)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
train_loader = DataLoader(train_data, batch_size, shuffle=True)
dev_loader = DataLoader(dev_data, batch_size, shuffle=False)
train_model(model, train_loader, dev_loader, optimizer, num_epochs)


Epoch 1/1, Train Loss: 3.8537, Dev Loss: 3.8522


In [197]:
# model_path = "parser_model_2.pth"
# torch.save(model.state_dict(), model_path)

In [202]:
# model_path = "parser_model_1.pth"
# model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

### 4) Implement the Inference Step (15 points)

In [203]:

def parse_sentence(model, sentence):

    stack = [('<ROOT>', '<ROOT>')]
    buffer = list(reversed(sentence))

    location=list(sentence)

    location.insert(0,('<ROOT>', '<ROOT>'))
    tree = [{} for _ in sentence]
    while len(stack) > 1 or len(buffer) > 0:

        feature_extractor = FeatureExtractor(DIMENSION, words_embedding, tags_embedding)
        features, _ = feature_extractor.extract_features(copy.copy(stack)    , copy.copy(buffer) )
        feature_tensor = torch.tensor(features, dtype=torch.float32).unsqueeze(0)  # add batch dimension
        scores = model(feature_tensor)
        legal_mask=get_legal_mask(stack, buffer,copy.copy(action_vocab))

        transition_idx = (scores +legal_mask ).argmax()
        transition = action_vocab[transition_idx]
        if transition[0] == 'shift' and buffer:
            stack.append(buffer.pop())
            
        elif transition[0]=='left_arc':
            
            s1=stack[-1]
            s2 = stack[-2]
           
            tree[location.index(s2)-1 ] = {
                'deprel': transition[1],
                'head': (location.index(s2), location.index(s1))
            }
            stack.pop(-2)
        elif transition[0]=='right_arc':
            s1=stack[-1]
            s2 = stack[-2] 
           
            tree[location.index(s1) -1] = {
                'deprel': transition[1],
                'head': (location.index(s1), location.index(s2))
            }
            stack.pop()
        if len(stack) == 1 and not buffer:
            break

    return tree

def get_legal_mask(stack, buffer,action_vocab):
    mask = torch.full_like(scores, float('-inf'))
    for i in range(len(action_vocab)):
        action=action_vocab[i]
        if len(stack)<2:
            if action==('shift', None) :
                mask[0][i] = 0
            
        else:
            if not action==('shift', None) :
                mask[0][i] = 0

    return mask



# 1	Ms.	_	_	NNP	_	2	compound	_	_
# 2	Haag	_	_	NNP	_	3	nsubj	_	_
# 3	plays	_	_	VBZ	_	0	root	_	_
# 4	Elianti	_	_	NNP	_	3	dobj	_	_
# 5	.	_	_	.	_	3	punct	_	_

# Example sentence
sentence = [('Ms.','NNP'), ('Haag','NNP'), ('plays','VBZ'), ('Elianti','NNP'),  ('.','.')]


parsed_tree = parse_sentence(model,sentence)
print("Parsed Dependency Tree:", parsed_tree)


Parsed Dependency Tree: [{'deprel': 'compound:prt', 'head': (1, 0)}, {'deprel': 'compound:prt', 'head': (2, 0)}, {'deprel': 'cc', 'head': (3, 0)}, {'deprel': 'nummod', 'head': (4, 0)}, {'deprel': 'cc', 'head': (5, 0)}]


In [207]:
predicted_trees = []
test_sentences=[]

for tree in test_trees:
    sentence=[]
    for i in range(1,len(tree.words())):
        sentence.append((tree.words()[i],tree.pos()[i]))

    test_sentences.append(sentence)

for processed_instance in test_sentences:
    parsed_tree = parse_sentence(model,processed_instance)
    predicted_trees.append(parsed_tree)

pprint(test_sentences[1])
print("----")
pprint(predicted_trees[1])

[('But', 'CC'),
 ('while', 'IN'),
 ('the', 'DT'),
 ('New', 'NNP'),
 ('York', 'NNP'),
 ('Stock', 'NNP'),
 ('Exchange', 'NNP'),
 ('did', 'VBD'),
 ("n't", 'RB'),
 ('fall', 'VB'),
 ('apart', 'RB'),
 ('Friday', 'NNP'),
 ('as', 'IN'),
 ('the', 'DT'),
 ('Dow', 'NNP'),
 ('Jones', 'NNP'),
 ('Industrial', 'NNP'),
 ('Average', 'NNP'),
 ('plunged', 'VBD'),
 ('190.58', 'CD'),
 ('points', 'NNS'),
 ('--', ':'),
 ('most', 'JJS'),
 ('of', 'IN'),
 ('it', 'PRP'),
 ('in', 'IN'),
 ('the', 'DT'),
 ('final', 'JJ'),
 ('hour', 'NN'),
 ('--', ':'),
 ('it', 'PRP'),
 ('barely', 'RB'),
 ('managed', 'VBD'),
 ('to', 'TO'),
 ('stay', 'VB'),
 ('this', 'DT'),
 ('side', 'NN'),
 ('of', 'IN'),
 ('chaos', 'NN'),
 ('.', '.')]
----
[{'deprel': 'cc', 'head': (1, 0)},
 {'deprel': 'cc', 'head': (2, 0)},
 {'deprel': 'cc', 'head': (3, 3)},
 {'deprel': 'cc', 'head': (4, 3)},
 {'deprel': 'cc', 'head': (5, 3)},
 {'deprel': 'cc', 'head': (6, 3)},
 {'deprel': 'cc', 'head': (7, 3)},
 {'deprel': 'cc', 'head': (8, 3)},
 {'deprel': 'cc', 

### 5) Evaluation (5 points)

In [206]:
test_trees_modified=[]
count=0
for tree in test_trees:

    tempt=[]
    for i in tree.deprels:
        tempt.append(({'head':(i,tree.deprels[i].head),'deprel':tree.deprels[i].deprel}))
       
    test_trees_modified.append(tempt)
       
pprint(test_trees_modified[0])

[{'deprel': 'discourse', 'head': (1, 7)},
 {'deprel': 'punct', 'head': (2, 7)},
 {'deprel': 'nsubj', 'head': (3, 7)},
 {'deprel': 'cop', 'head': (4, 7)},
 {'deprel': 'neg', 'head': (5, 7)},
 {'deprel': 'compound', 'head': (6, 7)},
 {'deprel': 'root', 'head': (7, 0)},
 {'deprel': 'punct', 'head': (8, 7)}]


In [209]:
def calculate_uas_las(gold_tree, predicted_tree):
    correct_heads = 0
    correct_labels = 0
    total = 0
    
    for gold_token, predicted_token in zip(gold_tree, predicted_tree):
        if not predicted_token:
            continue
        
        if gold_token['head'] == predicted_token['head']:
            correct_heads += 1
            if gold_token['deprel'] == predicted_token['deprel']:
                correct_labels += 1
        total += 1
    
    return correct_heads, correct_labels, total

total_correct_heads = 0
total_correct_labels = 0
total_tokens = 0

for gold_tree, predicted_tree in zip(test_trees_modified, predicted_trees):

    correct_heads, correct_labels, tokens = calculate_uas_las(gold_tree, predicted_tree)
    total_correct_heads += correct_heads
    total_correct_labels += correct_labels
    total_tokens += tokens

UAS_score = total_correct_heads / total_tokens
LAS_score = total_correct_labels / total_tokens

print(f"UAS (Unlabeled Attachment Score): {UAS_score:.4f}")
print(f"LAS (Labeled Attachment Score): {LAS_score:.4f}")


UAS (Unlabeled Attachment Score): 0.0326
LAS (Labeled Attachment Score): 0.0039
